In [1]:
!pip install "transformers==4.34.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.23.0" "bitsandbytes==0.41.1" "trl==0.4.7" "safetensors>=0.3.1" ipywidgets wandb --upgrade
!python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"
!pip install ninja packaging
!MAX_JOBS=1 pip install flash-attn --no-build-isolation

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# add parent directory to path
import sys
import os

project_dir = os.getcwd()
parent_dir = os.path.dirname(project_dir)
sys.path.insert(0, parent_dir)

In [6]:
# get data and convert to Datasets format
import pandas as pd
from datasets import Dataset, DatasetDict
from gaitor_function_calling.data.data_utils import DataAbstractor
from gaitor_function_calling.data.prompting_utils import INSTRUCTION

data_abstractor = DataAbstractor("op_glaive_10k.json", "full_sft")
# train, test = data_abstractor.build_data(INSTRUCTION, True, .997)
# data_abstractor.save(data_abstractor.raw_data, train, test)

train_df = pd.DataFrame(data_abstractor.train_data)
test_df = pd.DataFrame(data_abstractor.test_data)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9820
    })
    test: Dataset({
        features: ['text'],
        num_rows: 30
    })
})

In [3]:
# configs
from huggingface_hub import notebook_login
import wandb

model_id = "meta-llama/Llama-2-7b-chat-hf"
output_dir = "llama-7-chat-instruction-int4-fc-sft_fix"
hub_model_id = f"SebastianS/{output_dir}"
wandb.init(project="function_calling", id=output_dir, entity="sebastiansosa", notes="Trained for 3 epochs on chronologically ordered data with a train test split of 98% train and 2% test.")
wandb.define_metric("fc_combine", goal="maximize")
wandb.define_metric("exact_match", goal="maximize")
wandb.define_metric("perplexity", goal="minimize")
wandb.define_metric("epoch")
notebook_login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sebastiansosa. Use `wandb login --relogin` to force relogin


In [11]:
# get data and convert to Datasets format
import pandas as pd
from datasets import Dataset, DatasetDict
from gaitor_function_calling.data.data_utils import DataAbstractor
from gaitor_function_calling.data.prompting_utils import INSTRUCTION

data_abstractor = DataAbstractor("production_data-without_summary-only_fc.json", "sft")

train_df = pd.DataFrame(data_abstractor.train_data)
test_df = pd.DataFrame(data_abstractor.test_data)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

dataset_dict

No test data found


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 51885
    })
    test: Dataset({
        features: ['text'],
        num_rows: 17
    })
})

In [5]:
# setup metrics and evaluation
import json
from transformers import EvalPrediction
from gaitor_function_calling.evaluation.evaluation_utils import FunctionCallingMetric, compute_perplexity, get_logits_and_labels
from gaitor_function_calling.data.prompting_utils import INSTRUCTION, json_arguments_from_prompt
import numpy as np

fc_metric = FunctionCallingMetric()
def config_compute_metrics(tokenizer):
    def compute_metrics(pred: EvalPrediction):
        # pred.predictions are a batch of logits
        # pred.label_ids are a batch of tokens
        token_ids = np.argmax(pred.predictions, axis=-1)
        predictions = tokenizer.batch_decode(token_ids, skip_special_tokens=False)
        labels = tokenizer.batch_decode(pred.label_ids, skip_special_tokens=False)
        
        
    
        # Now use your metric class
        fc_result = fc_metric.run(predictions, labels)  # Implement this method in your class
    
        return {
            "fc_combine": fc_result
        }
    return compute_metrics

def custom_evaluation(eval_dataset, model, tokenizer, epoch, save_prompts_path=False):
    print("Starting custom evaluation.")
    model.eval()  # Set the model to evaluation mode
    results = {}

    fc_results = []
    exact_match_results = []
    perplexity_results = []


    if save_prompts_path:
        try:
            with open(save_prompts_path, "r") as f:
                dpo_data = json.load(f)
        except:
            dpo_data = []


    for idx, example in enumerate(eval_dataset):
        print(f"Example {idx}: ", end="")
        post_message = ""
        
        # Custom Function Calling metric
        prompts = None
        try:
            generated_arguments, expected_arguments, prompts = json_arguments_from_prompt(
                example["text"],
                model,
                tokenizer,
                INSTRUCTION,
                {"idx": idx, "epoch": epoch}
            )
            fc_result = fc_metric.run(generated_arguments, expected_arguments)

            fc_results.append(fc_result)

            if save_prompts_path:
                # add prompts to dpo_data.json
                dpo_data.append({
                    "fc_result": fc_result,
                    **prompts
                })
                with open(save_prompts_path, "w") as f:
                    json.dump(dpo_data, f)
        except Exception as e:
            post_message += f"Error function calling: {e}\n"
            fc_results.append(0)

        # exact match metric
        if prompts:
            exact_match_res = fc_metric._sentence_similarity(prompts["expected_str"].split("[/INST]")[1], prompts["generated_str"].split("[/INST]")[1])
            exact_match_results.append(exact_match_res)
        else:
            exact_match_results.append(0)

        # perplexity metric
        try:
            logits, labels = get_logits_and_labels(example["text"], model, tokenizer)
            perplexity = compute_perplexity(logits[..., :-1, :], labels).item()
            perplexity_results.append(perplexity)
        except Exception as e:
            post_message += f"Error perplexity: {e}\n"
            # perplexity_results.append(float('inf'))

        example_metric = {
            "fc_combine": fc_results[-1],
            "exact_match": exact_match_results[-1],
            "perplexity": perplexity_results[-1] if len(perplexity_results) else None
        }
        print(example_metric)
        if post_message:
            print(post_message)
        
    results["fc_combine"] =  sum(fc_results) / len(fc_results)
    results["exact_match"] =  sum(exact_match_results) / len(exact_match_results)
    results["perplexity"] =  sum(perplexity_results) / len(perplexity_results) if len(perplexity_results) else 1

    return results

In [6]:
# base LLM model and tokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
)
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# peft config on the model
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


In [8]:
# training config
from transformers import TrainingArguments
from trl import SFTTrainer


args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    report_to="wandb",
    push_to_hub=True,  # Enable pushing to Hub
    hub_model_id=hub_model_id,  # Hugging Face Hub model ID
    hub_strategy="every_save",  # Push to Hub every epoch
    disable_tqdm=True # disable tqdm since with packing values are in correct
)


max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_dict["train"],
    # eval_dataset=dataset_dict["test"],
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    compute_metrics=config_compute_metrics(tokenizer),
    dataset_text_field="text",
    args=args,
)

In [9]:
# Training and Evaluation Loop
eval_result = custom_evaluation(dataset_dict["test"], model, tokenizer, -1)
print(f"Evaluation results for epoch {-1}: {eval_result}")
wandb.log({"epoch": -1, **eval_result})

num_epochs = 3
for epoch in range(num_epochs):
    print(f"Begin epoch {epoch}")
    # Train for one epoch
    trainer.train()

    # Evaluate the model with the custom evaluation logic
    eval_result = custom_evaluation(dataset_dict["test"], model, tokenizer, epoch)

    wandb.log({"epoch": epoch, **eval_result})

    # Save the model after each epoch (optional)
    trainer.save_model()

    

    # Log or print evaluation results
    print(f"Evaluation results for epoch {epoch}: {eval_result}\n\n\n")

wandb.finish()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Starting custom evaluation.
Example 0: 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'fc_combine': 0.3577747344970703, 'exact_match': 0.8338859677314758, 'perplexity': 6.922008037567139}
Example 1: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 6.695792198181152}
Error function calling: 'NoneType' object has no attribute 'group'

Example 2: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 7.043546676635742}
Error function calling: 'NoneType' object has no attribute 'group'

Example 3: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 9.34947395324707}
Error function calling: No function call found in generated data

Example 4: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 9.34947395324707}
Error function calling: No function call found in generated data

Example 5: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 14.793754577636719}
Error function calling: No function call found in generated data

Example 6: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 14.793754577636719}
Error function calling: 'NoneType' object has no attribute 'group'

Example 7: {'fc

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.9875, 'learning_rate': 0.0002, 'epoch': 0.1}
{'loss': 0.7488, 'learning_rate': 0.0002, 'epoch': 0.19}
{'train_runtime': 206.082, 'train_samples_per_second': 4.042, 'train_steps_per_second': 0.505, 'train_loss': 0.8294446086883545, 'epoch': 0.24}
Starting custom evaluation.
Example 0: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 2.5376930236816406}
Error function calling: Extra data: line 1 column 53 (char 52)

Example 1: {'fc_combine': 0.7534257010980085, 'exact_match': 0.879702091217041, 'perplexity': 3.1918914318084717}
Example 2: {'fc_combine': 0.7213051075285132, 'exact_match': 0.8279994130134583, 'perplexity': 3.3109142780303955}
Example 3: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 2.889251470565796}
Error function calling: Extra data: line 1 column 3 (char 2)

Example 4: {'fc_combine': 0, 'exact_match': 0.677483081817627, 'perplexity': 2.889251470565796}
Error function calling: max() arg is an empty sequence

Example 5: {'fc_combine': 0, 'exact_match': 0, 

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▃▆█
exact_match,▁▄██
fc_combine,▁▂█▇
perplexity,█▂▁▁
train/epoch,▁▆█▁▆█▁▆█
train/global_step,▁▆████████████▆████████████▁████████████
train/learning_rate,▁▁▁▁▁▁
train/loss,█▅▄▃▂▁
train/total_flos,▁▁▁
train/train_loss,█▃▁
train/train_runtime,█▁▇
